<a href="https://colab.research.google.com/github/nicks165/VectorDatabases/blob/main/zilliz_wiki_2m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://zilliz.com/doc/integrate_with_cohere


https://zilliz.com/doc/quick_start

In [1]:
!pip install grpcio<=1.53.0
!pip install pymilvus cohere datasets
#!pip install --upgrade ipython

/bin/bash: =1.53.0: No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Initiatize Zilliz, create collection in code. Database created manually at https://cloud.zilliz.com/projects/MA==/databases

In [2]:
import cohere
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

#URI = "https://in01-8515adca5ddbe2b.aws-us-west-2.vectordb.zillizcloud.com:19543"
#USER = "user"
#PASSWORD = "deeplake_benchmarks"

URI = "https://in01-aa5649811ba8ff0.gcp-us-west1.vectordb.zillizcloud.com:443"
USER = "db_admin"
PASSWORD = "12qwaszxQW"

# Instantiate the client with the auth config
# Connect to Milvus Database
connections.connect(uri=URI, user=USER, password=PASSWORD, secure=True)

Load datset and get the embeddings from Cohere dataset at Huggingface

In [3]:
COLLECTION_NAME = "wiki_cohere_2m"
DIMENSION = 768

# Remove collection if it already exists
if utility.has_collection(COLLECTION_NAME):
    #utility.drop_collection(COLLECTION_NAME)
    collection = Collection(COLLECTION_NAME)
else:

    # Create collection which includes the id, title, and embedding.
    fields = [
      FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=True),
      FieldSchema(name='text', dtype=DataType.VARCHAR, max_length=20000),
      FieldSchema(name='vector', dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
    ]
    
    schema = CollectionSchema(fields=fields)
    collection = Collection(name=COLLECTION_NAME, schema=schema)

    # Create an AutoIndex index for collection.
    index_params = {
      'index_type': 'AUTOINDEX',
      'metric_type': 'IP',
      'params': {}
    }

    collection.create_index(field_name='vector', index_params=index_params)
    collection.load()

In [ ]:
from datasets import load_dataset
import torch
import cohere

#Load all documents + embeddings
limit = -1 # keep -1 for all, else update to a positive number to limit

max_docs_loaded = 0
docs_stream = load_dataset(f"Cohere/wikipedia-22-12-zh-embeddings", split="train")
docs = []
doc_embeddings = []

for doc in docs_stream:
    docs.append(doc)
    doc_embeddings.append(doc['emb'])
    max_docs_loaded += 1
    if (limit > 0 and max_docs_loaded == limit):
      break

Initialize index/collection (class in zilliz)

Confirm shape and save the dataset to disk

In [ ]:
import numpy as np

shape = np.array(doc_embeddings).shape
print(shape)

#docs_stream.save_to_disk("cohere dataset")

In [ ]:
def update_oneRecord(collection): 
  # initializing size of string
  N = 7
 
  # using random.choices()
  # generating random strings
  randomString = ''.join(random.choices(string.ascii_uppercase +
                             string.digits, k=N))
  
  dataToupd = "This is a new record " + randomString

  co = cohere.Client(f"o7lTEJeC1QHjU5I4Ee6U2I0m6l5wCOUPWqwoGM7H")  # Add your cohere API key from www.cohere.com
  newVector = {
      "vector": co.embed(texts=[dataToupd], model='multilingual-22-12').embeddings[0]
  }
  
  # Configure a batch process
  # create metadata dictionary
  properties = {
          "text": "New Data added " + dataToupd,
          }
  
  start_time = time.time() 
  #client.batch.add_data_object(properties, class_obj["class"], vector=newVector["vector"])


  data = [
        [properties['text']],
        [newVector['vector']]      
    ]

  collection.insert(data)

  print("Updated with one Record, Time taken --- {0} seconds ---".format(time.time() - start_time))

Querying the dataset

In [ ]:

def issueAndMeasureQueryTime(collection):

  co = cohere.Client(f"o7lTEJeC1QHjU5I4Ee6U2I0m6l5wCOUPWqwoGM7H")  # Add your cohere API key from www.cohere.com

  query1 = "What was the cause of the major recession in the early 20th century?"
  query2 = "Where is Mount Everest?"
  query3 = "something else"

  queries = [query1, query2, query3]

  timeTakenList = []

  ## Issue 3 queries and take the average
  for i in range(0, 3):
    # create the query embedding
    nearVector = {
        "vector": co.embed(texts=queries, model='multilingual-22-12').embeddings[i]
    }

    query_start_time = time.time()

    search_params = {
    "metric_type": "IP"
    }
    
    collection.load()

    results = collection.search(
      data = [nearVector['vector']],
      anns_field = "vector",
      param = search_params,
      limit = 10
    ) 
    
    query_end_time = time.time()

    timeTakenList.append(query_end_time - query_start_time)

    print(" For query number {0}, time taken for search = {1} ".format(i, timeTakenList[i]))

    #print(json.dumps(result, indent=4))
  
  averageTimeTaken = numpy.average(timeTakenList)
  print("Average time taken for search = {0} ".format(averageTimeTaken))

In [ ]:
def conditionalSearch(collection):
  
  co = cohere.Client(f"o7lTEJeC1QHjU5I4Ee6U2I0m6l5wCOUPWqwoGM7H")  # Add your cohere API key from www.cohere.com

  query1 = "What was the cause of the major recession in the early 20th century?"
  query2 = "Where is Mount Everest?"
  query3 = "something else"

  queries = [query1, query2, query3]

  timeTakenList = []

  ## Issue 3 queries and take the average

  for i in range(0, 3):
    # create the query embedding
    nearVector = {
        "vector": co.embed(texts=queries, model='multilingual-22-12').embeddings[i]
    }

    query_start_time = time.time()

    # using random.choices()
    # generating random strings
    #randomString = ''.join(random.choices(string.ascii_uppercase +
    #                         string.digits, k=10))

    search_params = {
    "metric_type": "IP"
    }

    collection.load()
    
    results = collection.search(
      data = [nearVector['vector']],
      anns_field = "vector",
      param = search_params,
      limit = 10,
      expr = "text like \"something random%\""
    ) 

    query_end_time = time.time()

    timeTakenList.append(query_end_time - query_start_time)

    print(" For query number {0}, time taken for conditional search = {1} ".format(i, timeTakenList[i]))

    #print(json.dumps(result, indent=4))
  
  averageTimeTaken = numpy.average(timeTakenList)
  print("Average time taken for conditional search = {0} ".format(averageTimeTaken))

Load embeddings and metadata into Zilliz

In [ ]:

# This is the main execution function

import time
import string
import random
import cohere
import numpy

batch_size=100

# Configure a batch process
for i in range(1000000, shape[0], batch_size):
  # start timer
  start_time = time.time()

  data = [
        [doc["text"] for doc in docs[i:i+batch_size]],
        doc_embeddings[i:i+batch_size]      
    ]

  collection.insert(data)

  

  # i+1 because index starts at 0 :)
  if(i+batch_size in [shape[0], batch_size, 1000, 10000, 100000, 500000, 1000000, 1500000, 2000000]):
    print(" For {0} entries, time taken for inserts = {1} ".format(i+batch_size, time.time() - start_time))

    # now get data for 1 single insert      
    update_oneRecord(collection)

    issueAndMeasureQueryTime(collection)

    conditionalSearch(collection)

# Flush records to make sure all rows are sent for indexing
if ((i % batch_size) == 0):
  collection.flush()
elif (i == (shape[0] - 1)):
  collection.flush()